In [1]:
import os
import sys

import numpy as np
import pandas as pd
import math

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import collections as coll
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

sys.path.append("../..")
import merge as merge

%matplotlib inline

In [2]:
TRAIN_PATH = '../../../input/train_1'

In [3]:

event_prefix = 'event000001003'
hits, cells, particles, truth = load_event(os.path.join(TRAIN_PATH, event_prefix))

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event_prefix, mem_bytes / 2**20))

event000001003 memory usage 15.95 MB


In [16]:
#labels_cone = pd.read_csv('../../event_1003_labels_train_cone.csv').label.values
#labels_helix = pd.read_csv('../../event_1000_labels_train_helix1.csv').label.values
labels_helix1 = pd.read_csv('../../loop_nofilter_noextend/event_1003_labels1_train_helix1.csv').label.values
labels_helix2 = pd.read_csv('../../loop_nofilter_noextend/event_1003_labels2_train_helix1.csv').label.values
labels_helix3 = pd.read_csv('../../loop_nofilter_noextend/event_1003_labels3_train_helix1.csv').label.values

In [ ]:
labels_hh = pd.read_csv('event_1003_helix2.csv').label.values

In [ ]:
#uniq_cone = np.unique(labels_cone)
uniq_helix = np.unique(labels_helix)
#print(uniq_cone)
print(uniq_helix)

In [5]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission


In [ ]:
one_submission = create_one_event_submission(1000, hits, labels_helix)
score = score_event(truth, one_submission)
print("helix score for event %d: %.8f" % (1000, score))
#one_submission = create_one_event_submission(1000, hits, labels_cone)
#score = score_event(truth, one_submission)
#print("cone score for event %d: %.8f" % (1000, score))

In [ ]:
labels_simple_merge = merge.merge_tracks(labels_helix, labels_cone)
one_submission = create_one_event_submission(1000, hits, labels_simple_merge)
score = score_event(truth, one_submission)
print("Simple merged cone+helix score for event %d: %.8f" % (1000, score))


In [ ]:
print('Removing cone outliers')
labels_c3 = np.copy(labels_cone)
labels_c3 = remove_outliers(labels_c3, hits, print_counts=True)

print('Removing helix outliers')
labels_h3 = np.copy(labels_helix)
labels_h3 = remove_outliers(labels_h3, hits, print_counts=True)

In [ ]:
print('Merging tracks')
tmp_labels = merge.heuristic_merge_tracks(labels_helix, labels_cone)
one_submission = create_one_event_submission(1000, hits, tmp_labels)
score = score_event(truth, one_submission)
print("cone score for event %d: %.8f" % (1000, score))

In [ ]:
print('Merging tracks')
tmp_labels = merge.heuristic_merge_tracks(labels_h3, labels_c3)
one_submission = create_one_event_submission(1000, hits, tmp_labels)
score = score_event(truth, one_submission)
print("Merged score for event %d: %.8f" % (1000, score))
# Baseline helix prior to outlier removal: 0.51217316
# Score: 0.4766-->0.4799 after improved outlier removal
# Score is 0.5037 if, after outlier removal,  we only add tracks from labels_c3 when no such track was recorded in labels_h3
# Score is 0.51087490 if we only remove bad volumes, duplicatez, and singletons, and only add tracks from labels_c3 when no such track was recorded in labels_h3
# Score is 0.51936460, remove badv, dupz, sings, selective track merging.
# Score is 0.52234651, remove badv, dupz, sings, selective track merging.
# Score is 0.52499622, remove badv, dupz, sings, selective track merging, overwrite smaller tracks of length <= 3.
# Score is 0.52653574, same as above except overwrite tracks of length <= 4
# Score is 0.52622554, same as above except overwrite tracks of length <= 5
# Score is 0.52209245, full outlier removal, overwrite tracks of length <= 4
# Score is 0.58658664 orig heuristic
#  --> 0.58240360 with aggressive outlier removal
#  --> 0.58621259 with aggressive cone removal, non-aggressive helix
#  --> 0.58417970 with aggressive helix removal, non-aggressive cone

In [8]:
# FIXME: Need to evaluate this better, seems to hurt!
def find_invalid_volumes(track, labels, df):
    invalid_ix = []

    hit_ix = np.where(labels == track)[0]
    df2 = df.loc[hit_ix]
    df2 = df2.sort_values('z_abs')
    hit_ix2 = df2.index.values
    all_positive = np.all(df2.z.values >= 0)
    all_negative = np.all(df2.z.values <= 0)
    volumes = df2.volume_id.values
    layers = df2.layer_id.values
    last_volume = volumes[0]
    last_layer = layers[0]
    # Tracks with the first volume of 8, 13, and 17 are very odd, sometimes
    # they hit in the negative way, sometimes the positive way,
    # sometimes a mix of both. Ignore these.
    if last_volume == 8 or last_volume == 13 or last_volume == 17:
        all_negative = False
        all_positive = False
    for idx, cur_vol in enumerate(volumes):
        cur_layer = layers[idx]
        if all_positive:
            # When we go from one volume to the next, we expect to see the
            # layer drop from a large layer id to a smaller layer id.
            # If we stay in the same volume, the layer id should not decrease.
            #if (last_volume != cur_vol and (cur_layer > (last_layer - 4))) or (last_volume == cur_vol and cur_layer < last_layer):
            if (last_volume == cur_vol and cur_layer < last_layer):
                invalid_ix.append(hit_ix2[idx])
            else:
                last_volume = cur_vol
                last_layer = cur_layer
        elif all_negative:
            # When we go from one volume to the next, we expect to see the
            # layer increase from a small layer id to a larger layer id.
            # If we stay in the same volume, the layer id should not increase.
            #if (last_volume != cur_vol and (cur_layer < (last_layer + 4))) or (last_volume == cur_vol and cur_layer > last_layer):
            if (last_volume == cur_vol and cur_layer > last_layer):
                invalid_ix.append(hit_ix2[idx])
            else:
                last_volume = volumes[idx]
                last_layer = layers[idx]
        else:
            last_volume = cur_vol
            last_layer = cur_layer

    return invalid_ix
    
def find_dimension_outlier(track, labels, df, dimension):
    outlier_ix = []
    hit_ix = np.where(labels == track)[0]

    # Need at least 3 values to determine if any look like outliers
    if len(hit_ix) < 3:
        return outlier_ix

    df2 = df.loc[hit_ix]        
    df2 = df2.sort_values('z')
    hit_ix2 = df2.index.values

    # Note, diff[0] is diff between 0 and 1
    diffs = np.diff(df2[dimension].values)

    growing_trend = 0
    shrinking_trend = 0
    for idx, diff in enumerate(diffs):
        if idx > 0 and diff > diffs[idx-1]:
            growing_trend = growing_trend + 1
        if idx > 0 and diff < diffs[idx-1]:
            shrinking_trend = shrinking_trend + 1

    check_largest_and_smallest = True
    if growing_trend > math.ceil(0.6*len(diffs)) or shrinking_trend > math.ceil(0.6*len(diffs)):
        check_largest_and_smallest = False

    if check_largest_and_smallest:
        # Find largest and smallest diffs, if largest is 20x larger than 2nd largest,
        # or smallest is 20x smaller than 2nd smallest, consider them outliers.
        top_two_ix = diffs.argsort()[-2:][::-1]
        large1 = diffs[top_two_ix[0]]
        large2 = diffs[top_two_ix[1]]
        bot_two_ix = diffs.argsort()[:2]
        small1 = diffs[bot_two_ix[0]]
        small2 = diffs[bot_two_ix[1]]

        largest_is_outlier = False
        smallest_is_outlier = False
        if large1 > 0 and large2 > 0 and large1 > 10.0 and large2 > 2.0 and (large2*7) < large1:
            largest_is_outlier = True
        if large1 < 0 and large2 < 0 and large1 < -10.0 and large2 < -2.0 and (large1*7) > large2:
            largest_is_outlier = True
        if small1 > 0 and small2 > 0 and small1 > 10.0 and small2 > 2.0 and (small2*7) < small1:
            smallest_is_outlier = True
        if small1 < 0 and small2 < 0 and small1 < -10.0 and small2 < -2.0 and (small1*7) > small2:
            smallest_is_outlier = True

        if largest_is_outlier or smallest_is_outlier:
            hit_ix_list = hit_ix.tolist()
            for idx, diff in enumerate(diffs):
                if (largest_is_outlier and diff == large1) or (smallest_is_outlier and diff == small1):
                    #print('Removing extreme outlier diff: ' + str(diff) + ', ix: ' + str(hit_ix2[idx + 1]) + ', from diffs: ' + str(diffs))
                    outlier_ix.append(hit_ix2[idx + 1])
                    hit_ix_list.remove(hit_ix2[idx + 1])

            # Re-generate the diffs now that we've removed the extreme outliers
            hit_ix = np.asarray(hit_ix_list)
            if len(hit_ix) < 3:
                return outlier_ix
            df2 = df.loc[hit_ix]        
            df2 = df2.sort_values('z')
            hit_ix2 = df2.index.values
            diffs = np.diff(df2[dimension].values)
                
    # Restrict to when the majority (75%+) of diffs are all in same direction
    neg_diffs = np.where(diffs < 0)[0]
    pos_diffs = np.where(diffs >= 0)[0]

    #print(df2[dimension].values)
    #print(hit_ix)
    #print('trk: ' + str(track) + ', diffs: ' + str(diffs))
    #print(neg_diffs)
    #print(pos_diffs)
    #print(df2)

    # Restrict to when the majority of diffs are either positive or negative.
    # (more difficult to detect outliers if diffs oscillate -ve and +ve)
    dim_vals = df2[dimension].values
    if len(neg_diffs) >= math.ceil(0.75*len(diffs)):
        # remove large positive ones.
        growing_trend = 0
        previous_diff = 0
        for idx, diff in enumerate(diffs):
            # Some tracks trend from negative to positive diffs, don't eliminate
            # positive values if it looks like we are trending that way.
            if idx > 0 and diff > diffs[idx-1]:
                growing_trend = growing_trend + 1
                if growing_trend > 2:
                    break
            else:
                growing_trend = 0
            # Use absolute value > 1.0 in case there is small delta each time,
            # we only try to find big jumps in the wrong direction.
            #print('nidx-' + dimension + ': ' + str(idx) + ', diff: ' + str(diff) + ', df ix: ' + str(hit_ix2[idx+1]))
            if diff > 1.0:
                # We sometimes see cases like:
                # diff[n-1] = -22
                # diff[n] = 12
                # diff[n+1] = -14
                # In this case, we want to remove n-1 as the outlier, since if that
                # was gone, diff[n] would be -10, which is more reasonable.
                # In cases where we see:
                # diff[0] = 23
                # diff[1] = -5
                # We want to check the dimension values directly instead of the diffs, it
                # could be that val[0] is the outlier.
                if idx == 0 and dim_vals[1] > dim_vals[2] and dim_vals[0] < dim_vals[2]:
                    # The real outlier is the first entry in the list.
                    outlier_ix.append(hit_ix2[0])
                elif idx == 0 or idx == (len(diffs)-1) or ((diff + diffs[idx-1]) > 0) or diffs[idx+1] > 0:
                    #print('Removing: ' + str(hit_ix2[idx+1]))
                    outlier_ix.append(hit_ix2[idx + 1])
                else:
                    # The real outlier is the previous one (i.e. diff[n-1] in the example above!
                    outlier_ix.append(hit_ix2[idx])
    
    elif len(pos_diffs) >= math.ceil(0.75*len(diffs)):
        # remove large negative ones
        shrinking_trend = 0
        for idx, diff in enumerate(diffs):
            # Some tracks trend from positive to negative diffs, don't eliminate
            # negative values if it looks like we are trending that way.
            if idx > 0 and diff < diffs[idx-1]:
                shrinking_trend = shrinking_trend + 1
                if shrinking_trend > 2:
                    break
            else:
                shrinking_trend = 0
            # Use absolute value > 1.0 in case there is small delta each time,
            # we only try to find big jumps in the wrong direction.
            #print('pidx-' + dimension + ': ' + str(idx) + ', diff: ' + str(diff) + ', df ix: ' + str(hit_ix2[idx+1]))
            if diff < -1.0:
                #print('Removing: ' + str(hit_ix2[idx+1]))
                # Similar to the negative case above, make sure we remove the real
                # outlier, in case the previous diff was misleading.
                if idx == 0 and dim_vals[1] < dim_vals[2] and dim_vals[0] > dim_vals[2]:
                    # The real outlier is the first entry in the list.
                    outlier_ix.append(hit_ix2[0])
                elif idx == 0 or idx == (len(diffs)-1) or ((diff + diffs[idx-1]) < 0) or diffs[idx+1] < 0:
                    #print('Removing: ' + str(hit_ix2[idx+1]))
                    outlier_ix.append(hit_ix2[idx + 1])
                else:
                    # The real outlier is the previous one (i.e. diff[n-1] in the example above!
                    outlier_ix.append(hit_ix2[idx])



    # Future ideas for patterns:
    # - average positive jump + average negative jump, for values that oscillate +ve and -ve
    # - absolute value of jump in same direction, this is hard since some tracks seem jumpy
    #   i.e. small diffs followed by a bigger jump, then smaller diffs. May need to tie that
    #   in with volume/layer/module ids, i.e. only allow bigger jumps between layers.
    return outlier_ix                

def find_duplicate_z(track, labels, df):
    def number_is_between(a1, a2, a3):
        return (a1 >= a2 and a2 >= a3) or (a1 <= a2 and a2 <= a3)

    def numbers_are_between(a1, a2, a3, b1, b2, b3):
        return number_is_between(a1, a2, a3) and number_is_between(b1, b2, b3)

    duplicatez_ix = []
    hit_ix = np.where(labels == track)[0]

    # Need at least 4 values to be able to evaluate duplicate z-values.
    if len(hit_ix) < 4:
        return duplicatez_ix

    df2 = df.loc[hit_ix]        
    df2 = df2.sort_values('z_abs')
    hit_ix2 = df2.index.values # remember new indexes after sorting
    xs = df2.x.values
    ys = df2.y.values
    zs = df2.z.values
    max_idx = len(zs) - 1

    z_counts = coll.Counter(df2.z.values).most_common(len(df2.z.values))

    if zs[0] == zs[1]:
        # zs at the beginning
        x1 = xs[2]
        x2 = xs[3]
        y1 = ys[2]
        y2 = ys[3]
        if numbers_are_between(xs[0], x1, x2, ys[0], y1, y2) and not numbers_are_between(xs[1], x1, x2, ys[1], y1, y2):
            # The first one is more consistent, delete the 2nd duplicate value
            duplicatez_ix.append(hit_ix2[1])
            #print('xs[1] ' + str(xs[1]) + ' <= x1 ' + str(x1) + ' <= x2 ' + str(x2))
            #print('ys[1] ' + str(ys[1]) + ' <= y1 ' + str(y1) + ' <= y2 ' + str(y2))
        elif numbers_are_between(xs[1], x1, x2, ys[1], y1, y2) and not numbers_are_between(xs[0], x1, x2, ys[0], y1, y2):
            # The second one is more consistent, delete the 1st duplicate value
            duplicatez_ix.append(hit_ix2[0])
            #print('b')
        elif numbers_are_between(xs[0], x1, x2, ys[0], y1, y2) and numbers_are_between(xs[1], x1, x2, ys[1], y1, y2):
            # Both z-values seem reasonable, need a tie-breaker to find out which is the right one.
            add_code_here = True
        # else, neither seem valid, unsure how to proceed, better off not rejecting any.

    if zs[-1] == zs[-2]:
        # zs at the end
        x1 = xs[-4]
        x2 = xs[-3]
        y1 = ys[-4]
        y2 = ys[-3]
        if numbers_are_between(x1, x2, xs[-2], y1, y2, ys[-2]) and not numbers_are_between(x1, x2, xs[-1], y1, y2, ys[-1]):
            # The first one is more consistent, delete the last duplicate value
            duplicatez_ix.append(hit_ix2[-1])
        elif numbers_are_between(x1, x2, xs[-1], y1, y2, ys[-1]) and not numbers_are_between(x1, x2, xs[-2], y1, y2, ys[-2]):
            # The last one is more consistent, delete the 1st duplicate value
            duplicatez_ix.append(hit_ix2[-2])
        elif numbers_are_between(x1, x2, xs[-1], y1, y2, ys[-1]) and numbers_are_between(x1, x2, xs[-2], y1, y2, ys[-2]):
            # Both z-values seem reasonable, need a tie-breaker to find out which is the right one.
            add_code_here = True
        # else, neither seem valid, unsure how to proceed, better off not rejecting any.
        
    # Idea: Find duplicate adjacent z-values. Remember x and y before and after the
    # duplicates. Choose z that lies between the two. If z at beginning or end,
    # need the two post (or pre-) x/y values to see the expected sign of the diff.

    # Note max_idx is largest valid index, we already handled the case where the
    # duplicate zs are at the beginning or end of the list.
    for idx in range(0, max_idx):
        if idx > 0 and (idx+2) <= max_idx and zs[idx] == zs[idx+1]:
            x1 = xs[idx-1]
            x2 = xs[idx+2]
            y1 = ys[idx-1]
            y2 = ys[idx+2]
            # now, x1 <= z1 <= x2, and y1 <= z1 <= y2
            if numbers_are_between(x1, xs[idx], x2, y1, ys[idx], y2) and not numbers_are_between(x1, xs[idx+1], x2, y1, ys[idx+1], y2):
                # The first one is more consistent, delete the 2nd duplicate value
                duplicatez_ix.append(hit_ix2[idx+1])
            elif numbers_are_between(x1, xs[idx+1], x2, y1, ys[idx+1], y2) and not numbers_are_between(x1, xs[idx], x2, y1, ys[idx], y2):
                # The second one is more consistent, delete the 1st duplicate value
                duplicatez_ix.append(hit_ix2[idx])
            elif numbers_are_between(x1, xs[idx], x2, y1, ys[idx], y2) and numbers_are_between(x1, xs[idx+1], x2, y1, ys[idx+1], y2):
                # Both z-values seem reasonable, need a tie-breaker to find out which is the right one.
                add_code_here = True
            # else, neither seem valid, unsure how to proceed, better off not rejecting any.

    #if z_counts[0][1] > 1:
    #    print('Duplicatez found on track ' + str(track) + ', removed: ' + str(duplicatez_ix))

    return duplicatez_ix

# TODO pi, -pi discontinuity 
def remove_track_outliers_slope(track, labels, hits, debug=False):
    final_outliers = []

    hhh_ix = np.where(labels == track)
    hhh_h = hits.loc[hhh_ix].sort_values('z')
    
    slopes_backward = []
    slopes_forward = []

    num_hits = len(hhh_h)
    # Only reliable with tracks >= 5 hits
    if num_hits < 5:
        return final_outliers

    if debug: print('backward:')
    for i in np.arange(num_hits-1,0,-1):
        a0 =  hhh_h.a0.values[i]
        a1 =  hhh_h.a0.values[i-1]
        r0 =  hhh_h.r.values[i]
        r1 =  hhh_h.r.values[i-1]
        if r0 == r1:
            r0 = r0 + 1e-8
        slope = (a0-a1)/(r0-r1) 
        slopes_backward.append(slope)
        if debug: print(hhh_h.hit_id.values[i], slope, a0)
        if i == 1:
            a0 = hhh_h.a0.values[0]
            a1 = hhh_h.a0.values[num_hits-1]
            r0 =  hhh_h.r.values[0]
            r1 =  hhh_h.r.values[num_hits-1]
            if r0 == r1:
                r0 = r0 + 1e-8
            slope = (a0-a1)/(r0-r1)
            slopes_backward.append(slope)
            if debug: print(hhh_h.hit_id.values[0], slope, a1)

    if debug: print('forward:')
    for i in np.arange(0,num_hits-1,1):
        a0 =  hhh_h.a0.values[i]
        a1 =  hhh_h.a0.values[i+1]
        r0 =  hhh_h.r.values[i]
        r1 =  hhh_h.r.values[i+1]
        if r0 == r1:
            r1 = r1 + 1e-8
        slope = (a1-a0)/(r1-r0) 
        slopes_forward.append(slope)
        if debug: print(hhh_h.hit_id.values[i], slope, a0)

        if i == num_hits-2:
            a0 = hhh_h.a0.values[0]
            a1 = hhh_h.a0.values[num_hits-1]
            r0 =  hhh_h.r.values[0]
            r1 =  hhh_h.r.values[num_hits-1]
            if r0 == r1:
                r1 = r1 + 1e-8
            slope = (a1-a0)/(r1-r0) 
            slopes_forward.append(slope)
            if debug: print(hhh_h.hit_id.values[num_hits-1], slope, a0)

    slopes_backward = np.asarray(slopes_backward)
    slopes_backward = np.reshape(slopes_backward, (-1, 1))
    slopes_forward = np.asarray(slopes_forward)
    slopes_forward = np.reshape(slopes_forward, (-1, 1))

    ss = StandardScaler()
    X_back = ss.fit_transform(slopes_backward)
    X_for = ss.fit_transform(slopes_forward)

    cl = DBSCAN(eps=0.0033, min_samples=1)
    outlier_labels_backward = cl.fit_predict(X_back)
    outlier_labels_forward = cl.fit_predict(X_for)

    if debug: print(outlier_labels_backward)
    if debug: print(outlier_labels_forward)

    track_counts = coll.Counter(outlier_labels_backward).most_common(1)
    most_common_id = track_counts[0][0]
    most_common_count = track_counts[0][1]

    outlier_indices_backward = []
    if most_common_count > 1 and len(np.unique(outlier_labels_forward)) < num_hits/2:
        for i in np.arange(num_hits-1,-1,-1):
            if outlier_labels_backward[i] != most_common_id:
                if debug: print(hhh_h.index.values[num_hits-1-i])
                outlier_indices_backward.append(hhh_h.index.values[num_hits-1-i])

    track_counts = coll.Counter(outlier_labels_forward).most_common(1)
    most_common_id = track_counts[0][0]
    most_common_count = track_counts[0][1]


    outlier_indices_forward = []
    if most_common_count > 1 and len(np.unique(outlier_labels_forward)) < num_hits/2:
        for i in np.arange(0,num_hits-1,1):
            if outlier_labels_forward[i] != most_common_id:
                if debug: print(hhh_h.index.values[i])
                outlier_indices_forward.append(hhh_h.index.values[i])


    outlier_candidates = list(set(outlier_indices_backward).intersection(outlier_indices_forward))


    if debug: print('before removal:' + str(outlier_candidates))

    for i in range(len(outlier_candidates)):
        candidate = hhh_h.loc[outlier_candidates[i]]
        found = False
        for index, row in hhh_h.iterrows():
            if np.absolute(candidate.z-row.z) == 0.5 and candidate.volume_id == row.volume_id \
            and candidate.layer_id == row.layer_id and candidate.module_id != row.module_id:
                # true hits
                if debug: print('true hit' + str(outlier_candidates[i]))
                found = True
        if found is False:
            final_outliers.append(outlier_candidates[i])

    if debug: print('new loutliers:' + str(final_outliers))

    # If we determine that half (or more) of the hits need to be removed, we may have messed
    # up, so do not return any outliers.
    max_removal_threshold = math.floor(num_hits/2)
    if len(final_outliers) >= max_removal_threshold:
        final_outliers = []

    return final_outliers

    
def remove_track_outliers(track, labels, hits, aggressive):
    labels = np.copy(labels)
    found_bad_volume = 0
    found_bad_dimension = 0
    found_bad_slope = 0
    found_bad_z = 0

    # Check if the sorted hits (on z-axis) go through the volumes
    # and layers in the expected order
    bad_volume_ix = find_invalid_volumes(track, labels, hits)
    if len(bad_volume_ix) > 0:
        #print('track ' + str(track) + ' bad volume: ' + str(bad_volume_ix))
        found_bad_volume = found_bad_volume + len(bad_volume_ix)
        for bvix in bad_volume_ix:
            labels[bvix] = 0

    if True:
        # Check if the sorted hits (on z-axis) go through the volumes
        # and layers in the expected order
        duplicatez_ix = find_duplicate_z(track, labels, hits)
        if len(duplicatez_ix) > 0:
            #print('track ' + str(track) + ' duplicate z: ' + str(duplicatez_ix))
            found_bad_z = found_bad_z + len(duplicatez_ix)
            for bzix in duplicatez_ix:
                labels[bzix] = 0

    if True:
        # Check the helix slope, discard hits that do not match
        outlier_slope_ix = remove_track_outliers_slope(track, labels, hits)
        if len(outlier_slope_ix) > 0:
            #print('track ' + str(track) + ' slope outliers: ' + str(outlier_slope_ix))
            found_bad_slope = found_bad_slope + len(outlier_slope_ix)
            for oix in outlier_slope_ix:
                labels[oix] = 0

    if aggressive:
        # Next analysis, from remaining hits, sort by 'z' (roughly time-based),
        # check for anomolies in other dimensions.
        outlier_ix = find_dimension_outlier(track, labels, hits, 'y')
        if len(outlier_ix) > 0:
            #print('track ' + str(track) + ' outlier dimension y: ' + str(outlier_ix))
            found_bad_dimension = found_bad_dimension + len(outlier_ix)
            for oix in outlier_ix:
                labels[oix] = 0

        # Next analysis, from remaining hits, sort by 'z' (roughly time-based),
        # check for anomolies in z dimensions (i.e. outliers at beginning/end)
        outlier_ix = find_dimension_outlier(track, labels, hits, 'z')
        if len(outlier_ix) > 0:
            #print('track ' + str(track) + ' outlier dimension z: ' + str(outlier_ix))
            found_bad_dimension = found_bad_dimension + len(outlier_ix)
            for oix in outlier_ix:
                labels[oix] = 0
            
    return (labels, found_bad_volume, found_bad_dimension, found_bad_z, found_bad_slope)

def remove_outliers(labels, hits, aggressive=False, print_counts=True):
    tracks = np.unique(labels)
    hits['z_abs'] = hits.z.abs()
    hits['r'] = np.sqrt(hits.x**2+hits.y**2)
    hits['a0'] = np.arctan2(hits.y,hits.x)
    count_rem_volume = 0
    count_rem_dimension = 0
    count_duplicatez = 0
    count_rem_slope = 0
    count_singletons = 0
    for track in tracks:
        if track == 0:
            continue
        track_hits = np.where(labels == track)[0]
        if len(track_hits) > 3:
            (labels, c1, c2, c3, c4) = remove_track_outliers(track, labels, hits, aggressive)
            count_rem_volume = count_rem_volume + c1
            count_rem_dimension = count_rem_dimension + c2
            count_duplicatez = count_duplicatez + c3
            count_rem_slope = count_rem_slope + c4

    # Remove singletons, we do not get any score for those. This is done
    # last, in case removing the outliers (above) removed enough hits
    # from a track to create a new singleton.
    tracks = np.unique(labels)
    for track in tracks:
        if track == 0:
            continue
        track_hits = np.where(labels == track)[0]
        if len(track_hits) == 1:
            count_singletons = count_singletons + 1
            labels[track_hits[0]] = 0

    if print_counts:
        print('Total removed due to bad volumes: ' + str(count_rem_volume))
        print('Total removed due to bad dimensions: ' + str(count_rem_dimension))
        print('Total removed due to duplicate zs: ' + str(count_duplicatez))
        print('Total removed due to bad slopes: ' + str(count_rem_slope))
        print('Total removed singleton hits: ' + str(count_singletons))

    return labels

In [ ]:
labels_h3 = np.copy(labels_helix)

#one_submission = create_one_event_submission(1000, hits, labels_h3)
#score = score_event(truth, one_submission)
#print("Initial score before outlier removal for event %d: %.8f" % (1000, score))

labels_h3 = remove_outliers(labels_h3, hits)
one_submission = create_one_event_submission(1000, hits, labels_h3)
score = score_event(truth, one_submission)
print("Initial score after outlier removal for event %d: %.8f" % (1000, score))

# before score: 0.57485796
# after old outlier removal: 0.57471551
# after slope removal: 0.56906582 (1006 removed due to bad slopes)
# after slope removal + threshold: 0.56966146 (962 removed due to bad slopes)

In [ ]:
# Play around with the track number to find a track that contains multiple particles
hhh_ix = np.where(labels_helix == 5944)
print(hhh_ix)
hhh_t = truth.loc[hhh_ix]
hhh_t = hhh_t.sort_values('tz')
print(hhh_t)

# Now add r/a0 as columns to the hits, and see if there's a way to detect
# which hits belong to the track, and which should be considered outliers.
hits['r'] = np.sqrt(hits.x**2+hits.y**2)
hits['a0'] = np.arctan2(hits.y,hits.x)
hhh_h = hits.loc[hhh_ix]


#hhh_h = hhh_h[(hhh_h.hit_id != 15459) & (hhh_h.hit_id != 16190) & (hhh_h.hit_id != 16155) ]

hhh_h = hhh_h.sort_values('z')
print(hhh_h)

In [ ]:
# Dupz -> only remove when value ends in .0000 or .5000?
# 59828: a: 0,025628, r: 13,745506, slope=a/r=0,001864463920062, 59825: a: 0,025181, r: 13,590172, slope=0,001852883098168
# 67285: r: -35,053009, a: -0,072102, slope=0,002056941816322, 67284: r: -46,909088, a: -0,088433, slope=0,001885199729315
# 59036: r: 25,316536, a: 0,041827, slope=0,001652161259344, 59007: r: 11,899605, a: 0,019226, 0,001615683881944
# Ones that we had trouble with for slope outlier detection:
#x 11/6 hits - track 1373 outliers: [24207, 39572, 39574, 48338, 78104, 78108] should remove [24207, 39572, 39574]
#y 17/7 hits - track 1644 outliers: [21896, 29838, 36816, 89394, 89395, 111614, 95550] should remove: [89393, 89395]
# 19/7 hits - track 1698 outliers: [80388, 73708, 42159, 73717, 86837, 86838, 41657] should remove: [86838, 86387, 80388, 73708, 42159]
# 14/5 hits - track 1902 outliers: [23136, 37256, 116299, 30509, 116560] should remove: [116561]
#x 15/9 hits - track 1963 outliers: [30432, 37195, 43502, 23025, 75475, 111219, 111222, 37180, 30367] should remove: [111222]
#y 13/5 hits - track 2390 outliers: [76962, 84260, 84264, 45130, 45486] should remove: [45128, 45130, 45486, 76962, 84260, 84264, 91275]
# --> actually, really good! this was a very complicated case.
#x 20/11 hits - track 2445 outliers: [70818, 65315, 69259, 67852, 67858, 69236, 105080, 70776, 66589, 69246, 69247] should remove: [67852, 69246, 69236, 69259, 70776, 71464]
# 16/6 hits - track 2522 outliers: [44685, 44686, 83245, 76114, 21534, 76126] should remove:  [21534, 21672, 44209, 44686, 76114, 83245, 90582]
# 15/5 hits - track 3417 outliers: [28458, 81457, 81464, 80985, 35775] should remove: [81457, 80985]
#x 15/8 hits - track 3427 outliers: [101154, 39335, 98506, 120427, 24496, 119921, 31986, 84925] should remove: []
#y 15/6 hits - track 4225 outliers: [119013, 118535, 31339, 45003, 76853, 38453] should remove: [118535]
#y 13/5 hits - track 4318 outliers: [84482, 45732, 77481, 77200, 77203] should remove: [77200, 77481, 84482, 84707, 119929]
# 18/5 hits - track 5047 outliers: [44323, 44330, 44331, 30926, 37591] should remove: [22142, 30926, 37591, 44319, 44331, 44323, 44330]
# 13/5 hits - track 5101 outliers: [117442, 22595, 22833, 111923, 95867] should remove   [117444]
#xx 13/6 hits - track 5420 outliers: [86917, 92518, 86918, 92523, 108468, 108469] should remove [86918]
#y 23/10 hits - track 5422 outliers: [94144, 81504, 88577, 94531, 74492, 88165, 43052, 110419, 21500, 110718] should remove [21650, 21512, 43052, 43060, 74492, 81504, 88165, 110718, 88577, 94531]
# 17/5 hits - track 5826 outliers: [43176, 89005, 94905, 88988, 88991] should remove [43170, 43176, 89005, 88991, 88988, 94905, 94878, 111049]
# 14/5 hits - track 6244 outliers: [34257, 34706, 34708, 19508, 34264] should remove [79229, 72675, 34708, 34264, 34706, 34697, 19718, 19508]
#y 12/5 hits - track 6409 outliers: [35969, 73987, 28709, 73990, 42508] should remove [109342, 109340, 93542, 73987]
# 15/6 hits - track 6740 outliers: [38990, 32046, 39374, 23633, 23666, 38997] should remove [23633, 23666, 32046, 38997, 39374]

In [7]:
def safe_outlier_removal(labels, hits, truth):
    tracks = np.unique(labels)
    hits['z_abs'] = hits.z.abs()
    hits['r'] = np.sqrt(hits.x**2+hits.y**2)
    hits['a0'] = np.arctan2(hits.y,hits.x)
    count_removed = 0
    count_not_removed = 0
    for track in tracks:
        if track == 0:
            continue
        track_hits = np.where(labels == track)[0]
        if len(track_hits) > 3:
            outlier_slope_ix = remove_track_outliers_slope(track, labels, hits)
            if len(outlier_slope_ix) > 0:
                tdf = truth.loc[track_hits]
                truth_count = coll.Counter(tdf.particle_id.values).most_common(1)
                truth_particle_id = truth_count[0][0]
                for out_ix in outlier_slope_ix:
                    if tdf.loc[out_ix].particle_id != truth_particle_id:
                        labels[out_ix] = 0
                        count_removed = count_removed + 1
                    else:
                        count_not_removed = count_not_removed + 1

    print('safe count_removed: ' + str(count_removed))
    print('safe count_not_removed: ' + str(count_not_removed))
    return labels

def perfect_outlier_removal(labels, hits, truth):
    tracks = np.unique(labels)
    count_removed = 0
    count_not_removed = 0
    for track in tracks:
        if track == 0:
            continue
        track_hits = np.where(labels == track)[0]
        if len(track_hits) > 3:
            tdf = truth.loc[track_hits]
            truth_count = coll.Counter(tdf.particle_id.values).most_common(1)
            truth_particle_id = truth_count[0][0]
            for hit_ix in track_hits:
                if tdf.loc[hit_ix].particle_id != truth_particle_id:
                    labels[hit_ix] = 0
                    count_removed = count_removed + 1
                else:
                    count_not_removed = count_not_removed + 1

    print('perfect count_removed: ' + str(count_removed))
    print('perfect count_not_removed: ' + str(count_not_removed))
    return labels

In [ ]:
labels_h2 = np.copy(labels_helix)
labels_h2 = remove_outliers(labels_h2, hits)
labels_h3 = np.copy(labels_helix)
labels_h3 = safe_outlier_removal(labels_h3, hits, truth)
labels_h4 = np.copy(labels_helix)
labels_h4 = perfect_outlier_removal(labels_h4, hits, truth)

one_submission = create_one_event_submission(1000, hits, labels_helix)
score = score_event(truth, one_submission)
print("Initial score for event %d: %.8f" % (1000, score))

one_submission = create_one_event_submission(1000, hits, labels_h2)
score = score_event(truth, one_submission)
print("Score after normal outlier removal for event %d: %.8f" % (1000, score))

one_submission = create_one_event_submission(1000, hits, labels_h3)
score = score_event(truth, one_submission)
print("Score after safe outlier removal score for event %d: %.8f" % (1000, score))

one_submission = create_one_event_submission(1000, hits, labels_h4)
score = score_event(truth, one_submission)
print("Score after perfect outlier removal for event %d: %.8f" % (1000, score))

In [ ]:
labels_hh2 = np.copy(labels_hh)
labels_hh2 = remove_outliers(labels_hh2, hits)
labels_hh3 = np.copy(labels_hh)
labels_hh3 = safe_outlier_removal(labels_hh3, hits, truth)
labels_hh4 = np.copy(labels_hh)
labels_hh4 = perfect_outlier_removal(labels_hh4, hits, truth)

one_submission = create_one_event_submission(1003, hits, labels_hh)
score = score_event(truth, one_submission)
print("Initial score for event %d: %.8f" % (1003, score))

one_submission = create_one_event_submission(1003, hits, labels_hh2)
score = score_event(truth, one_submission)
print("Score after normal outlier removal for event %d: %.8f" % (1003, score))

one_submission = create_one_event_submission(1003, hits, labels_hh3)
score = score_event(truth, one_submission)
print("Score after safe outlier removal score for event %d: %.8f" % (1003, score))

one_submission = create_one_event_submission(1003, hits, labels_hh4)
score = score_event(truth, one_submission)
print("Score after perfect outlier removal for event %d: %.8f" % (1003, score))

In [17]:
labels_h1 = np.copy(labels_helix1)
labels_h2 = np.copy(labels_helix2)
labels_h3 = np.copy(labels_helix3)
labels_merged = merge.heuristic_merge_tracks(labels_h1, labels_h2)
labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_h3)
one_submission = create_one_event_submission(1003, hits, labels_merged)
score = score_event(truth, one_submission)
print("Merge score no removal for event %d: %.8f" % (1003, score))

labels_hh1 = np.copy(labels_helix1)
labels_hh2 = np.copy(labels_helix2)
labels_hh3 = np.copy(labels_helix3)
labels_hh1 = remove_outliers(labels_hh1, hits)
labels_hh2 = remove_outliers(labels_hh2, hits)
labels_hh3 = remove_outliers(labels_hh3, hits)
labels_merged2 = merge.heuristic_merge_tracks(labels_hh1, labels_hh2)
labels_merged = merge.heuristic_merge_tracks(labels_merged2, labels_hh3)
one_submission = create_one_event_submission(1003, hits, labels_merged2)
score = score_event(truth, one_submission)
print("Merge score outlier removal for event %d: %.8f" % (1003, score))



Simple replacement of unclassified hits: 0
Similar tracks (no-op): 5598
Multiple non-trivial tracks: 5493
--> of which partial track ID 0 hits were updated: 0
--> of which partial track ID non-0 hits were updated: 15074
Tracks to be lengthened: 5693
--> of which track ID 0 hits were updated: 0
--> of which labels2 unique track lengths were: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Simple replacement of unclassified hits: 0
Similar tracks (no-op): 9320
Multiple non-trivial tracks: 2954
--> of which partial track ID 0 hits were updated: 0
--> of which partial track ID non-0 hits were updated: 4835
Tracks to be lengthened: 5437
--> of which track ID 0 hits were updated: 0
--> of which labels2 unique track lengths were: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 19]
Merge score no removal for event 1003: 0.55489967
Total removed due to bad volumes: 0
Total removed due to bad dimensions: 0
Total removed due to duplicate zs: 223
Total removed due to bad slopes: 422
Total

In [ ]:
labels_merge1 = merge.heuristic_merge_tracks(labels_helix, labels_hh)
one_submission = create_one_event_submission(1003, hits, labels_merge1)
score = score_event(truth, one_submission)
print("Merge score no removal for event %d: %.8f" % (1003, score))

labels_merge2 = merge.heuristic_merge_tracks(labels_h2, labels_hh2)
one_submission = create_one_event_submission(1003, hits, labels_merge2)
score = score_event(truth, one_submission)
print("Merge score normal outlier removal for event %d: %.8f" % (1003, score))

labels_merge3 = merge.heuristic_merge_tracks(labels_h3, labels_hh3)
one_submission = create_one_event_submission(1003, hits, labels_merge3)
score = score_event(truth, one_submission)
print("Merge score safe removal for event %d: %.8f" % (1003, score))

labels_merge4 = merge.heuristic_merge_tracks(labels_h4, labels_hh4)
one_submission = create_one_event_submission(1003, hits, labels_merge4)
score = score_event(truth, one_submission)
print("Merge score perfect removal for event %d: %.8f" % (1003, score))

In [ ]:
labels_q1 = np.copy(labels_helix)
labels_q1 = remove_outliers(labels_q1, hits)

labels_q2 = np.copy(labels_hh)
labels_q2 = remove_outliers(labels_q2, hits)

labels_mergeq1 = merge.heuristic_merge_tracks(labels_q1, labels_q2)
one_submission = create_one_event_submission(1003, hits, labels_mergeq1)
score = score_event(truth, one_submission)
print("Merge score normal no-slope removal for event %d: %.8f" % (1003, score))


In [ ]:
# TODO pi, -pi discontinuity 
def remove_track_outliers_slope(track, labels, hits, debug=False):
    hhh_ix = np.where(labels == track)
    hhh_h = hits.loc[hhh_ix].sort_values('z')
    
    slopes_backward = []
    slopes_forward = []


    num_hits = len(hhh_h)
    if debug: print('backward:')
    for i in np.arange(num_hits-1,0,-1):
        a0 =  hhh_h.a0.values[i]
        a1 =  hhh_h.a0.values[i-1]
        r0 =  hhh_h.r.values[i]
        r1 =  hhh_h.r.values[i-1]
        if r0 == r1:
            r0 = r0 + 1e-8
        slope = (a0-a1)/(r0-r1) 
        slopes_backward.append(slope)
        if debug: print(hhh_h.hit_id.values[i], slope, a0)
        if i == 1:
            a0 = hhh_h.a0.values[0]
            a1 = hhh_h.a0.values[num_hits-1]
            r0 =  hhh_h.r.values[0]
            r1 =  hhh_h.r.values[num_hits-1]
            if r0 == r1:
                r0 = r0 + 1e-8
            slope = (a0-a1)/(r0-r1)
            slopes_backward.append(slope)
            if debug: print(hhh_h.hit_id.values[0], slope, a1)

    if debug: print('forward:')
    for i in np.arange(0,num_hits-1,1):
        a0 =  hhh_h.a0.values[i]
        a1 =  hhh_h.a0.values[i+1]
        r0 =  hhh_h.r.values[i]
        r1 =  hhh_h.r.values[i+1]
        if r0 == r1:
            r1 = r1 + 1e-8
        slope = (a1-a0)/(r1-r0) 
        slopes_forward.append(slope)
        if debug: print(hhh_h.hit_id.values[i], slope, a0)

        if i == num_hits-2:
            a0 = hhh_h.a0.values[0]
            a1 = hhh_h.a0.values[num_hits-1]
            r0 =  hhh_h.r.values[0]
            r1 =  hhh_h.r.values[num_hits-1]
            if r0 == r1:
                r1 = r1 + 1e-8
            slope = (a1-a0)/(r1-r0) 
            slopes_forward.append(slope)
            if debug: print(hhh_h.hit_id.values[num_hits-1], slope, a0)

    slopes_backward = np.asarray(slopes_backward)
    slopes_backward = np.reshape(slopes_backward, (-1, 1))
    slopes_forward = np.asarray(slopes_forward)
    slopes_forward = np.reshape(slopes_forward, (-1, 1))

    ss = StandardScaler()
    X_back = ss.fit_transform(slopes_backward)
    X_for = ss.fit_transform(slopes_forward)

    cl = DBSCAN(eps=0.0033, min_samples=1)
    outlier_labels_backward = cl.fit_predict(X_back)
    outlier_labels_forward = cl.fit_predict(X_for)

    if debug: print(outlier_labels_backward)
    if debug: print(outlier_labels_forward)

    track_counts = coll.Counter(outlier_labels_backward).most_common(1)
    most_common_id = track_counts[0][0]
    most_common_count = track_counts[0][1]

    outlier_indices_backward = []
    if most_common_count > 1 and len(np.unique(outlier_labels_forward)) < num_hits/2:
        for i in np.arange(num_hits-1,-1,-1):
            if outlier_labels_backward[i] != most_common_id:
                if debug: print(hhh_h.index.values[num_hits-1-i])
                outlier_indices_backward.append(hhh_h.index.values[num_hits-1-i])

    track_counts = coll.Counter(outlier_labels_forward).most_common(1)
    most_common_id = track_counts[0][0]
    most_common_count = track_counts[0][1]


    outlier_indices_forward = []
    if most_common_count > 1 and len(np.unique(outlier_labels_forward)) < num_hits/2:
        for i in np.arange(0,num_hits-1,1):
            if outlier_labels_forward[i] != most_common_id:
                if debug: print(hhh_h.index.values[i])
                outlier_indices_forward.append(hhh_h.index.values[i])


    outlier_candidates = list(set(outlier_indices_backward).intersection(outlier_indices_forward))
    final_outliers = []

    if debug: print('before removal:' + str(outlier_candidates))

    for i in range(len(outlier_candidates)):
        candidate = hhh_h.loc[outlier_candidates[i]]
        found = False
        for index, row in hhh_h.iterrows():
            if np.absolute(candidate.z-row.z) == 0.5 and candidate.volume_id == row.volume_id \
            and candidate.layer_id == row.layer_id and candidate.module_id != row.module_id:
                # true hits
                if debug: print('true hit' + str(outlier_candidates[i]))
                found = True
        if found is False:
            final_outliers.append(outlier_candidates[i])

    if debug: print('new loutliers:' + str(final_outliers))

    return final_outliers


def remove_outliers_slope(labels, hits):
    tracks = np.unique(labels)
    count_rem_volume = 0
    count_rem_dimension = 0
    count_duplicatez = 0
    count_singletons = 0
    for track in tracks:
        if track == 0:
            continue
        track_hits = np.where(labels == track)[0]
        if len(track_hits) > 4:
            outliers = remove_track_outliers_slope(track, labels, hits)
            if len(outliers) > 0:
                do_something = True
                # filter out the outliers
            
    return labels

In [ ]:
labels_h3 = np.copy(labels_helix)
outliers = remove_track_outliers_slope(14, labels_h3, hits)
print('Slope outliers: ' + str(outliers))

In [ ]:
print('Removing helix outliers')
labels_h3 = np.copy(labels_helix)
labels_h3 = remove_outliers_slope(labels_h3, hits)

In [ ]:
# Play around with the track number to find a track that contains multiple particles
hhh_ix = np.where(labels_helix == 14)
print(hhh_ix)
hhh_t = truth.loc[hhh_ix]
hhh_t = hhh_t.sort_values('tz')
print(hhh_t)

# Now add r/a0 as columns to the hits, and see if there's a way to detect
# which hits belong to the track, and which should be considered outliers.
hits['r'] = np.sqrt(hits.x**2+hits.y**2)
hits['a0'] = np.arctan2(hits.y,hits.x)
hhh_h = hits.loc[hhh_ix]


#hhh_h = hhh_h[(hhh_h.hit_id != 15459) & (hhh_h.hit_id != 16190) & (hhh_h.hit_id != 16155) ]

hhh_h = hhh_h.sort_values('z')
print(hhh_h)

In [ ]:
print('Removing cone outliers')
labels_c3 = np.copy(labels_cone)
labels_c3 = remove_outliers_slope(labels_c3, hits)

print('Removing helix outliers')
labels_h3 = np.copy(labels_helix)
labels_h3 = remove_outliers_slope(labels_h3, hits)

print('Merging tracks')
tmp_labels = merge.heuristic_merge_tracks(labels_h3, labels_c3)
one_submission = create_one_event_submission(1000, hits, tmp_labels)
score = score_event(truth, one_submission)
print("Merged score for event %d: %.8f" % (1000, score))

#Merged score for event 1000: 0.58701361

In [ ]:

labels_c2 = np.copy(labels_cone)
labels_c2 = remove_outliers(labels_c2, hits)

labels_simple_merge = merge.merge_tracks(labels_helix, labels_c2)
one_submission = create_one_event_submission(1000, hits, labels_simple_merge)
#one_submission = create_one_event_submission(1000, hits, labels_c2)
score = score_event(truth, one_submission)
print("Outlier removal score for cone event %d: %.8f" % (1000, score))
# Orig Score no removals: 0.26152679
# With outlier removal:
# volume: 1231
# bad dim: 6733
# score: 0.18735180
# With volume removal: 3520, score: 0.23268937
# With volume removal, treat -ve+8--> +ve: 2841, score: 0.23394451
# With ignore of 8, otherwise full checks: 1056, score: 0.25499520
# With ignore of 8, light checks: 1, score: 0.26152679
# Bad volume removal: 1, Bad dims: 222, score: 0.25587569, merged: 0.48291748
# Bad volume removal: 1, Bad dims: 220, duplicatez: 1675, score: 0.25657711, merged: 0.48742414
# HELIX: Bad vol: 15, score: 0.51204521
# HELIX: Bad vol: 15, duplicatez: 399, score: 0.51065321
# HELIX: Bad vol: 15, duplicatez: 399, bad dim: 286, score: 0.49635631
# HELIX: Bad vol: 15, duplicatez: 399, bad dim: 1518, score: 0.50364853
# CONE: Bad vol: 0, duplicatez: 1675, bad dim: 1239, score: 0.25871803, merged: 0.48569633

In [ ]:
tmp_labels = merge.heuristic_merge_tracks(labels_helix, labels_c2)
one_submission = create_one_event_submission(1000, hits, tmp_labels)
score = score_event(truth, one_submission)
print("cone score for event %d: %.8f" % (1000, score))

In [ ]:
labels_h2 = np.copy(labels_helix)
labels_h2 = remove_outliers(labels_h2, hits)#, aggressive=True)

In [ ]:
track_to_remove = 157
labels_h3 = np.copy(labels_helix)
xxx_ix = np.where(labels_h3 == track_to_remove)[0]
print('xxx_ix: ' + str(xxx_ix))
(labels_h3x, _, _, _) = remove_track_outliers(track_to_remove, labels_h3, hits, False)#, True)
xxx_ixx = np.where(labels_h3x == track_to_remove)[0]
print('xxx_ixx: ' + str(xxx_ixx))
#(labels_h3, cx, cy, cz) = remove_track_outliers(track_to_remove, labels_c3, hits)
#labels_c3x = np.copy(labels_c3)
#one_submission = create_one_event_submission(1000, hits, labels_c3)
#score = score_event(truth, one_submission)
#print("Outlier removal score for cone event %d: %.8f" % (1000, score))
# rem 0,0, score 0.26152679
#xxx_ix = np.where(labels_c3 == track_to_remove)[0]
#print('xxx_ix: ' + str(xxx_ix))
xxx_df = truth.loc[xxx_ix]
xxx_df = xxx_df.sort_values('tz')
xxx_df

In [ ]:
track_to_remove = 157
labels_h3 = np.copy(labels_helix)
xxx_ix = np.where(labels_h3 == track_to_remove)[0]
print('xxx_ix: ' + str(xxx_ix))
(labels_h3x, _, _, _) = remove_track_outliers(track_to_remove, labels_h3, hits, False)#, True)
xxx_ixx = np.where(labels_h3x == track_to_remove)[0]
print('xxx_ixx: ' + str(xxx_ixx))
#(labels_h3, cx, cy, cz) = remove_track_outliers(track_to_remove, labels_c3, hits)
#labels_c3x = np.copy(labels_c3)
#one_submission = create_one_event_submission(1000, hits, labels_c3)
#score = score_event(truth, one_submission)
#print("Outlier removal score for cone event %d: %.8f" % (1000, score))
# rem 0,0, score 0.26152679
#xxx_ix = np.where(labels_c3 == track_to_remove)[0]
#print('xxx_ix: ' + str(xxx_ix))
xxx_df = hits.loc[xxx_ix]
xxx_df = xxx_df.sort_values('z')
xxx_df

In [ ]:
tr2_df = truth.loc[truth['particle_id'] == 306250134780379136]
tr2_df = tr2_df.sort_values('tz')
tr2_df

In [ ]:
tr2_df = truth.loc[truth['particle_id'] == 49542619558051840]
tr2_df = tr2_df.sort_values('tz')
tr2_df

In [ ]:
helidx = np.where(labels_helix==7095)[0]
print(helidx)

In [ ]:
one_submission = create_one_event_submission(1000, hits, labels_cone)
score = score_event(truth, one_submission)
print("Outlier removal score for cone event %d: %.8f" % (1000, score))

In [ ]:
#labels_c2 = np.copy(labels_cone)
#trks2 = np.unique(labels_c2)
#count_rem_volume = 0
#count_rem_dimension = 0
#for trk2 in trks2:
#    if trk2 == 0:
#        continue
#    trk2_hits = np.where(labels_c2 == trk2)[0]
#    if len(trk2_hits) > 3:
#        (labels_c2, c1, c2) = remove_track_outliers(trk2, labels_c2, hits)
#        count_rem_volume = count_rem_volume + c1
#        count_rem_dimension = count_rem_dimension + c2

#print('Total removed due to bad volumes: ' + str(count_rem_volume))
#print('Total removed due to bad dimensions: ' + str(count_rem_dimension))

track_to_remove = 63542
labels_c3 = np.copy(labels_cone)
xxx_ix = np.where(labels_c3 == track_to_remove)[0]
print('xxx_ix: ' + str(xxx_ix))
(labels_c3, cx, cy, cz) = remove_track_outliers(track_to_remove, labels_c3, hits)
labels_c3x = np.copy(labels_c3)
one_submission = create_one_event_submission(1000, hits, labels_c3)
score = score_event(truth, one_submission)
print("Outlier removal score for cone event %d: %.8f" % (1000, score))
# rem 0,0, score 0.26152679
xxx_ix = np.where(labels_c3 == track_to_remove)[0]
print('xxx_ix: ' + str(xxx_ix))
xxx_df = hits.loc[xxx_ix]
xxx_df = xxx_df.sort_values('z')
# Bad indexes: 59855, 61697
print(xxx_df)
xxx_ix = np.where(labels_cone == 63542)
xxx_df = hits.loc[xxx_ix]
xxx_df = xxx_df.sort_values('z')
# indexes in question: ???
xxx_df

In [ ]:
labels_helix = merge.renumber_labels(labels_helix)
max_track = np.amax(labels_helix)
labels_cone[labels_cone != 0] = labels_cone[labels_cone != 0] + max_track

In [ ]:
labelsx = np.copy(labels_cone)
trackx = 63949
outx = find_dimension_outlier(trackx, labelsx, hits, 'y')
print('outx: ' + str(outx))

In [ ]:
xxx_ix = np.where(labels_cone == 63542)
xxx_df = hits.loc[xxx_ix]
xxx_df = xxx_df.sort_values('z')
# indexes in question: ???
xxx_df

In [ ]:
tr_df = truth.loc[xxx_ix]
tr_df = tr_df.sort_values('tz')
# BAD: 59855, 61697
# sorted z, see steps for 'y', ensure they are consistent (good for 59855, ok for 61697)
# --> if most in same direction except one/two outliers, and outlier magnitude is large....
all_y1 = tr_df.ty.values
all_y = np.diff(all_y1)
print('mean: ' + str(all_y.mean()))
print('max: ' + str(all_y.max()))
print('min: ' + str(all_y.min()))
#[49193, 59886, 61710]
tr_df

In [ ]:
tr2_df = truth.loc[truth['particle_id'] == 166642325903114240]
tr2_df = tr2_df.sort_values('tz')
tr2_df

In [ ]:
track 150 outlier dimension y: [100706]
trk large -: 160, large1: -1.592, large2: -61.533
track 160 outlier dimension y: [41261]
track 169 outlier dimension y: [17867]
track 181 outlier dimension y: [103556]
track 188 outlier dimension y: [661]
track 194 outlier dimension y: [40343]
trk large -: 208, large1: -1.2258, large2: -30.6159
track 208 outlier dimension y: [23693]
trk large -: 223, large1: -1.3697, large2: -36.1324
track 223 outlier dimension y: [29081]
trk large +: 263, large1: 38.0123, large2: 1.7837
track 263 outlier dimension y: [19765]
trk large +: 281, large1: 102.727, large2: 3.78752
track 281 outlier dimension y: [100657]
track 286 outlier dimension y: [62478]

In [ ]:
hhh_ix = np.where(labels_helix == 5549)
hhh_df = truth.loc[hhh_ix]
hhh_df = hhh_df.sort_values('tz')
# indexes in question: ???
hhh_df

In [ ]:
tr_df = truth.loc[hhh_ix]
tr_df = tr_df.sort_values('tz')
# BAD: 59855, 61697
# sorted z, see steps for 'y', ensure they are consistent (good for 59855, ok for 61697)
# --> if most in same direction except one/two outliers, and outlier magnitude is large....
all_y1 = tr_df.ty.values
all_y = np.diff(all_y1)
print('mean: ' + str(all_y.mean()))
print('max: ' + str(all_y.max()))
print('min: ' + str(all_y.min()))
#[48366, 48517, 51460, 54360, 54471, 57091, 57133]
#[6596, 6539, 4232, 4180]
tr_df

In [ ]:
track_x = np.array([
    [
        [-12.166300, -156.817993, 962.000],
        [20.349701, -242.162003, 1498.500],
        [48.221401, -284.333008, 1795.500],
        [48.529099, -284.743988, 1798.500],
        [122.458000, -404.139008, 2554.500]
    ],
    [
        [-7.17245, -44.973099, -1098.0],
        [-6.67366, -39.278999, -962.5],
        [-6.65474, -39.091202, -958.0],
        [-6.01885, -33.464001, -822.5],
        [-5.99555, -33.277901, -818.0]
    ],
    [
        [-6.829120, 30.917900, 40.626202],
        [-28.525200, 169.742004, 212.766998],
        [-34.125198, 258.694000, 325.647003],
        [-31.138201, 361.954987, 453.306000],
        [-12.355800, 503.367004, 629.344971]
    ]
])

track_y = np.array([
    [1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1],
    [0, 0, 1, 1, 1]
])
print(track_x.shape)

In [ ]:
tracks = np.unique(labels_h3)
track_x = np.zeros((len(tracks), 10, 3))
track_y = np.zeros((len(tracks), 10, 1))
max_xval = np.amax(hits.x.values)
print(max_xval)
max_yval = np.amax(hits.y.values)
print(max_yval)
max_zval = np.amax(hits.z.values)
print(max_zval)
for idx, track in enumerate(tracks):
    if track == 0:
        continue
    hit_ix = np.where(labels_h3 == track)[0]
    df2 = hits.loc[hit_ix]
    df2 = df2.sort_values('z_abs')
    hit_ix2 = df2.index.values # remember new indexes after sorting
    #print(df2)
    xs = df2.x.values
    ys = df2.y.values
    zs = df2.z.values

    # From this track, figure out the most common particle ID from the truth.
    # Any hits from our track that belong to that particle will be set to '1'
    # (correctly predicted hit), otherwise '0' (outlier).
    tf2 = truth.loc[hit_ix]
    counters = coll.Counter
    track_counts = coll.Counter(tf2.particle_id.values).most_common(len(hit_ix2))
    track_particle_id = track_counts[0][0]

    for i in range(len(hit_ix2)):
        if i < 10:
            track_x[idx][i][0] = xs[i] / max_xval
            track_x[idx][i][1] = ys[i] / max_yval
            track_x[idx][i][2] = zs[i] / max_zval
            if (truth.loc[hit_ix2[i]].particle_id == track_particle_id):
                track_y[idx][i][0] = 3
            else:
                track_y[idx][i][0] = 30
            #track_y[idx][i][0] = (truth.loc[hit_ix2[i]].particle_id == track_particle_id)
            
    #if idx < 10:
    #    ignore_it = True
    #elif idx < 20:
    #    print(tf2)
    #    print(df2)
    #    print(track_x[idx])
    #    print(track_y[idx])
    #else:
    #    break




In [ ]:
import keras.layers as L
import keras.models as M

# The inputs to the model.
# We will create two data points, just for the example.
data_x = np.array([
    # Datapoint 1
#    [
#        # Input features at timestep 1
#        [1, 2, 3],
#        # Input features at timestep 2
#        [4, 5, 6]
#    ],
    # Datapoint 2
    [
        # Features at timestep 1
        [7, 8, 9],
        # Features at timestep 2
        [10, 11, 12]
    ]
])

# The desired model outputs.
# We will create two data points, just for the example.
data_y = np.array([
    # Datapoint 1
    # Target features at timestep 2
#    [105, 106, 107, 108, 109],
    # Datapoint 2
    # Target features at timestep 2
    [205, 206, 207, 208, 209]
])

# Each input data point has 2 timesteps, each with 3 features.
# So the input shape (excluding batch_size) is (2, 3), which
# matches the shape of each data point in data_x above.
model_input = L.Input(shape=(10, 3))

# This RNN will return timesteps with 4 features each.
# Because return_sequences=True, it will output 2 timesteps, each
# with 4 features. So the output shape (excluding batch size) is
# (2, 4), which matches the shape of each data point in data_y above.
#model_output = L.LSTM(4, return_sequences=False)(model_input)
#model_output = L.LSTM(100, return_sequences=True)(model_input)
#model_output = L.Dense(100, activation='linear')(model_output)
#model_output = L.Dense(4, activation='linear')(model_output)
#model_output = L.LSTM(4, return_sequences=False)(model_output)
model_output = L.Bidirectional(L.LSTM(10, return_sequences=True, stateful=False))(model_input)
#model_output = L.LSTM(30, return_sequences=True, stateful=False)(model_output)
model_output = L.Bidirectional(L.LSTM(10, return_sequences=True, stateful=False))(model_output)
model_output = L.TimeDistributed(L.Dense(1, activation='linear'))(model_output)
#model_output = L.Dense(100, activation='linear')(model_output)
#model_output = L.Dense(30, activation='linear')(model_output)
# Create the model.
model = M.Model(inputs=model_input, outputs=model_output)

# You need to pick appropriate loss/optimizers for your problem.
# I'm just using these to make the example compile.
model.compile(optimizer='sgd', loss='mean_squared_error')
#model.compile(loss='binary_crossentropy', optimizer='adam')

# Train
#model.fit(data_x, data_y)
#model.fit(track_x, track_y)

# batch_size=3
# num_steps=5


In [ ]:

#for e in range(100):
#    #for i in range(track_x.shape[0]):
#    #    tx = np.expand_dims(track_x[i], axis=0)
#    #    ty = np.expand_dims(track_y[i], axis=0)
model.fit(track_x, track_y, batch_size=1, epochs=5, verbose=1)
#for i in range(track_x.shape[0]):
#        tx = np.expand_dims(track_x[i], axis=0)

for i in range(track_x.shape[0]):
    tx = np.expand_dims(track_x[i], axis=0)
    yhat = model.predict(tx)
    ty = track_y[i]
    print(ty)
    print(yhat)
    if i > 10:
        break
# old - (3 LSTM Layers): Loss: 0.0810
# old - (2 LSTM, 2 Dense (100, 30)): Loss ~ 0.08
# New TimeDistributed Loss: 0.2072
# New normalized TimeDistributed loss: 0.3196
# New normalized Bidi-LSTM TimeDistributed loss: 0.3100, 0.1920, 0.1362, 0.1273, 0.1193
#  -> same, but with 10-hit track input: 0.5413, 0.4646, 0.4242, 0.3943, 0.3550
#  -> 3 for right hit, 10 for outlier, 0 for ignore: 5.1141, 5.0738, 5.0602, 5.0038, 4.6757,
#                                                    4.1964, 3.9969, 3.8963, 3.8313, 3.7737
#  -> 3 for right hit, 30 for outlier, 0 for ignore: 62.5914

In [ ]:
for i in range(track_x.shape[0]):
    if i < 1000:
        continue
    tx = np.expand_dims(track_x[i], axis=0)
    yhat = model.predict(tx)
    ty = track_y[i]
    print('Prediction: ' + str(i))
    print(ty)
    print(yhat)
    if i > 1020:
        break

In [ ]:
print(hits.head())
print(len(hits))

print(particles.head())
print(len(particles))

print(cells.head())
print(len(cells))

print(truth.head())
print(len(truth))

In [ ]:
# From the XY plane
g = sns.jointplot(hits.x, hits.y, size=12)

#Clear the axes containing the scatter plot
g.ax_joint.cla()
# Set the current axis to the parent of ax
plt.sca(g.ax_joint)

volumes = hits.volume_id.unique()
for volume in volumes:
    v = hits[hits.volume_id == volume]
    # scattering the hit coordinates with the particle size = 1
    plt.scatter(v.x, v.y, s=1, label='volume {}'.format(volume))

plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

In [ ]:
#From the YZ plane
g = sns.jointplot(hits.z, hits.y, s=1, size=12)
g.ax_joint.cla()
plt.sca(g.ax_joint)

volumes = hits.volume_id.unique()
for volume in volumes:
    v = hits[hits.volume_id == volume]
    plt.scatter(v.z, v.y, s=1, label='volume {}'.format(volume))

plt.xlabel('Z (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

In [ ]:
# From XYZ 3D perspective
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')
for volume in volumes:
    v = hits[hits.volume_id == volume]
    ax.scatter(v.z, v.x, v.y, s=1, label='volume {}'.format(volume), alpha=0.5)
ax.set_title('SHit Locations')
ax.set_xlabel('Z (millimeters)')
ax.set_ylabel('X (millimeters)')
ax.set_zlabel('Y (millimeters)')
plt.show()


In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.distplot(particles.nhits.values, axlabel='Hits/Particle', bins=50)
plt.title('Distribution of number of hits per particle for event 1000.')
plt.subplot(1, 2, 2)
plt.pie(particles.groupby('q')['vx'].count(),
        labels=['negative', 'positive'],
        autopct='%.0f%%',
        shadow=True,
        radius=1)
plt.title('Distribution of particle charges.')
plt.show()


In [ ]:
# Visualize the original particles of tracks, most particle collisions are generated from the origin

g = sns.jointplot(particles.vz, particles.vy,  s=3, size=12)
g.ax_joint.cla()
plt.sca(g.ax_joint)

n_hits = particles.nhits.unique()
for n_hit in n_hits:
    p = particles[particles.nhits == n_hit]
    plt.scatter(p.vz, p.vy, s=1, label='Hits {}'.format(n_hit))

plt.xlabel('Z (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(122, projection='3d')
ax = fig.add_subplot(121, projection='3d')

for charge in [-1, 1]:
    q = particles[particles.q == charge]
    ax.scatter(q.vz, q.vx, q.vy, s=1, label='Charge {}'.format(charge), alpha=0.5)
ax.set_title('Sample of 1000 Particle initial location')
ax.set_xlabel('Z (millimeters)')
ax.set_ylabel('X (millimeters)')
ax.set_zlabel('Y (millimeters)')
ax.legend()
plt.show()

In [ ]:
HIT_COUNT = 12
particle1 = particles.loc[particles.nhits == HIT_COUNT].iloc[0]
particle2 = particles.loc[particles.nhits == HIT_COUNT].iloc[1]
particle3 = particles.loc[particles.nhits == HIT_COUNT].iloc[2]


p_traj_surface1 = truth[truth.particle_id == particle1.particle_id][['tx', 'ty', 'tz']]
p_traj_surface2 = truth[truth.particle_id == particle2.particle_id][['tx', 'ty', 'tz']]
p_traj_surface3 = truth[truth.particle_id == particle3.particle_id][['tx', 'ty', 'tz']]



p_traj1 = (p_traj_surface1
          .append({'tx': particle1.vx, 'ty': particle1.vy, 'tz': particle1.vz}, ignore_index=True)
          .sort_values(by='tz'))

p_traj2 = (p_traj_surface2
          .append({'tx': particle2.vx, 'ty': particle2.vy, 'tz': particle2.vz}, ignore_index=True)
          .sort_values(by='tz'))

p_traj3 = (p_traj_surface3
          .append({'tx': particle3.vx, 'ty': particle3.vy, 'tz': particle3.vz}, ignore_index=True)
          .sort_values(by='tz'))


# Visualize XY projection to the Z-axis

plt.plot(p_traj1.tz, p_traj1.ty, '-o', label='hits')
plt.plot(p_traj2.tz, p_traj2.ty, '-o', label='hits')
plt.plot(p_traj3.tz, p_traj3.ty, '-o', label='hits')
plt.xlabel('Z (mm)')
plt.ylabel('Y (mm)')
plt.title('ZY projection to the X-axis')
plt.legend()
plt.show()





fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')

ax.plot(
    xs=p_traj1.tx,
    ys=p_traj1.ty,
    zs=p_traj1.tz, marker='o')
ax.plot(
    xs=p_traj2.tx,
    ys=p_traj2.ty,
    zs=p_traj2.tz, marker='o')
ax.plot(
    xs=p_traj3.tx,
    ys=p_traj3.ty,
    zs=p_traj3.tz, marker='o')



ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('Z  (mm) -- Detection layers')
plt.title('Trajectories of two particles as they cross the detection surface ($Z$ axis).')
plt.show()